In [ ]:
import os
import getpass

import langgraph
from langgraph.graph import StateGraph
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import llm

from typing import TypedDict, Optional

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [13]:
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.2)

In [4]:
prompt = PromptTemplate.from_template(
    """
You are a flight assistant for United Airlines helping customers experiencing flight disruptions. 

Extract structured context from the user's message.

User message: {user_input}

Return JSON with the following fields:
- intent: one of ["rebook", "cancel", "check_status", "compensation", "hotel", "other"]
- last_name: string
- confirmation code: alphanumeric string
- flight_number: optional string
- destination_city: optional string
- departure_city: optional string
- travel_date: optional string (YYYY-MM-DD)
- sentiment: one of ["calm", "frustrated", "angry", "panicked"]
- customer_priority: one of ["basic", "silver", "gold", "platinum", "unknown"]
- escalation_risk: boolean
"""
)

In [ ]:
context_extraction_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [ ]:
def extract_context_node(state):
    user_input = state["user_input"]
    response = context_extraction_chain.run({"user_input": user_input})
    state["context"] = response
    return state

In [ ]:
class ContextState(TypedDict):
    user_input: str
    context: Optional[str]

workflow = StateGraph(ContextState)
workflow.add_node("extract_context", extract_context_node)
workflow.set_entry_point("extract_context")
workflow.set_finish_point("extract_context")
context_graph = workflow.compile()

In [24]:
initial_state = {
    "user_input": "My flight to Denver was just canceled. I’m flying United and I have gold status."
}

result = context_graph.invoke(initial_state)
print(result["context"])

/var/folders/mf/8c9q6j514k1_fgpgs5221sch0000gn/T/ipykernel_79764/2430716936.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = context_extraction_chain.run({"user_input": user_input})


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]